# Catchment Classification

In [1]:
import pandas as pd

import async_retriever as ar
import hydrosignatures as hsg
import pygeohydro as hgh
import pynhd
from pygeohydro import NWIS
from pynhd import NLDI, StreamCat

In this tutorial, we use HyRiver libraries to classify catchments based on their hydrological characteristics. We use the 531 USGS stations that was used in [Newman et al. (2017)](https://doi.org/10.1175/JHM-D-16-0284.1). Then we get their catchments attributes from [StreamCat](https://www.epa.gov/national-aquatic-resource-surveys/streamcat-dataset) to carry out the classification. We use hydrological signatures based on streamflow observations that we obtain from NWIS.

Let's start by getting station IDs and retrieve their streamflow observations. We can either use `pygeohydro.get_camels` function or directly obtain the IDs from the link below.

In [2]:
url = "/".join(
    (
        "https://gist.githubusercontent.com/cheginit",
        "229c83c89eee3801a586bcb3ebb4e825/raw/newman_ids.txt",
    )
)
camels_ids = ar.retrieve_text([url])[0].split(",")

Next, we use `pygeohydro.NWIS` to obtain streamflow observations for these stations for 2000-2020 period. We set `to_xarray=True` since not only we get streamflow data, but also stations' metadata are returned.

In [3]:
start, end = "2000-01-01", "2020-12-31"
nwis = NWIS()
qobs = nwis.get_streamflow(camels_ids, (start, end), to_xarray=True)

Among these stations, there are some that have considerable missing data. We use `pygeohydro.streamflow_fillna` to filter out stations with more than five days of missing data per year and, for the remaining stations, it fills their missing data with their day-of-year average over the period of record.

In [4]:
qobs_df = hgh.streamflow_fillna(qobs["discharge"].to_pandas(), 5)
station_ids = qobs_df.columns

For retrieving watershed characteristics of these stations, first we need to find their associated NHDPlus ComID so we can pass to `pynhd.streamcat`. We use `pynhd.NLDI` to do so.

In [5]:
nldi = NLDI()
comids = nldi.getfeature_byid("nwissite", station_ids)

StreamCat contains around 600 characteristics from a variety of sources. Since, we are interested in physical characteristics of these watersheds, first we need to filter out those characteristics that are not related. To do so, we use `StreamCat.metrics_df`. Note that, in PyNHD, the main function for querying StreamCat is `streamcat`. There is also a class called `StreamCat` that contains useful information about the dataset such as available options for characteristic (called metrics in StreamCat), their descriptions, and so on. So, it's always a good idea to start with instantiating this class and figure out the available options.

In [6]:
sc = StreamCat()
sc.metrics_df.head()

,AOI,DATE_DOWNLOADED,FINAL_TABLE,INDICATOR_CATEGORY,METADATA,METRIC_DESCRIPTION,METRIC_NAME,METRIC_UNITS,SOURCE_NAME,SOURCE_URL,UUID,YEAR,SLOPE
0,"Cat, Ws",02-13-2016,Dams,Anthropogenic,https://edg.epa.gov/metadata/rest/document?id=...,Total possible volume of all reservoirs (NID_S...,DamNIDStor,cubic meters/square kilometer,USGS NAWQA,ftp://ftpext.usgs.gov/pub/er/va/reston/NAWQA_E...,46EB14C8-FC84-4BA8-9C7F-CC4938FD1FF3,NaN,NaN
1,"Cat, Ws",02-13-2016,Dams,Anthropogenic,https://edg.epa.gov/metadata/rest/document?id=...,Normal (most common) volume of all reservoirs ...,DamNrmStor,cubic meters/square kilometer,USGS NAWQA,ftp://ftpext.usgs.gov/pub/er/va/reston/NAWQA_E...,46EB14C8-FC84-4BA8-9C7F-CC4938FD1FF3,NaN,NaN
2,"Cat, Ws",NaN,Elevation,Natural,https://edg.epa.gov/metadata/rest/document?id=...,Mean AOI elevation in meters.,Elev,Meters,NHDPlusV2,http://www.horizon-systems.com/NHDPlus/NHDPlus...,1730F0BC-7019-4821-8B31-4A6E7B3DA625,NaN,NaN
3,"Cat, Ws",04-16-2015,GeoChemPhys1,Natural,https://edg.epa.gov/metadata/rest/document?id=...,Mean % of lithological ferric oxide (Fe2O3) co...,Fe2O3,Percent,USGS Sciencebase,https://www.sciencebase.gov/catalog/folder/534...,46EB14C8-FC84-4BA8-9C7F-CC4938FD1FF3,NaN,NaN
4,"Cat, Ws",03-17-2016,AgriculturalNitrogen,Anthropogenic,https://edg.epa.gov/metadata/rest/document?id=...,Mean rate of synthetic nitrogen fertilizer app...,Fert,mean rate kilogram Nitrogen/hectare/year,EnviroAtlas,https://enviroatlas.epa.gov/enviroatlas/DataFa...,46EB14C8-FC84-4BA8-9C7F-CC4938FD1FF3,NaN,NaN


We narrow down the list of metrics, by first looking at `FINAL_TABLE` column of the metrics' dataframe.

By going over these options, we select those that are related to physical characteristics of the catchments.

In [7]:
themes = (
    "Dams",
    "Elevation",
    "Kffact",
    "BFI",
    "STATSGO_Set1",
    "NLCD",
    "STATSGO_Set2",
    "PRISM_0809",
    "Runoff",
    "Precip_Minus_EVT",
    "WetIndex",
)
metrics = sc.metrics_df[sc.metrics_df.FINAL_TABLE.isin(themes)]
metrics.head()

,AOI,DATE_DOWNLOADED,FINAL_TABLE,INDICATOR_CATEGORY,METADATA,METRIC_DESCRIPTION,METRIC_NAME,METRIC_UNITS,SOURCE_NAME,SOURCE_URL,UUID,YEAR,SLOPE
0,"Cat, Ws",02-13-2016,Dams,Anthropogenic,https://edg.epa.gov/metadata/rest/document?id=...,Total possible volume of all reservoirs (NID_S...,DamNIDStor,cubic meters/square kilometer,USGS NAWQA,ftp://ftpext.usgs.gov/pub/er/va/reston/NAWQA_E...,46EB14C8-FC84-4BA8-9C7F-CC4938FD1FF3,NaN,NaN
1,"Cat, Ws",02-13-2016,Dams,Anthropogenic,https://edg.epa.gov/metadata/rest/document?id=...,Normal (most common) volume of all reservoirs ...,DamNrmStor,cubic meters/square kilometer,USGS NAWQA,ftp://ftpext.usgs.gov/pub/er/va/reston/NAWQA_E...,46EB14C8-FC84-4BA8-9C7F-CC4938FD1FF3,NaN,NaN
2,"Cat, Ws",NaN,Elevation,Natural,https://edg.epa.gov/metadata/rest/document?id=...,Mean AOI elevation in meters.,Elev,Meters,NHDPlusV2,http://www.horizon-systems.com/NHDPlus/NHDPlus...,1730F0BC-7019-4821-8B31-4A6E7B3DA625,NaN,NaN
7,"Cat, Ws",04-01-2016,Kffact,Natural,https://edg.epa.gov/metadata/rest/document?id=...,Mean soil erodibility (Kf) factor (unitless) o...,AgKffact,mean surface soil erodibility factors,Penn State Soil Info,http://www.soilinfo.psu.edu/index.cgi?soil_dat...,EBB945ED-0151-4DBF-8FE0-806165A58973,NaN,NaN
9,"Cat, Ws",03-10-2015,BFI,Natural,https://edg.epa.gov/metadata/rest/document?id=...,Baseflow is the component of streamflow that c...,BFI,Percent,USGS Water Mission Area,http://water.usgs.gov/GIS/metadata/usgswrd/XML...,66C0ED41-2707-4732-A906-E9D89E8F5A6B,NaN,NaN


By looking at these metrics, we realize that some metrics are annual summaries and require specifying the year. We can use `StreamCat.valid_years` to check the available years. For example, let's get the last year of the available years for each metric. Note that, we need to replace `[Year]` in `METRIC_NAME` column of `metrics_df`.

In [8]:
names = metrics[metrics.YEAR.isna()].METRIC_NAME.to_list()
names_yr = metrics[~metrics.YEAR.isna()].METRIC_NAME
names += [n.replace("[Year]", str(sc.valid_years[n][-1])) for n in names_yr]

Now, we use these metrics and ComIDs to obtain their characteristics.

In [9]:
attrs = pynhd.streamcat(names, comids=comids.comid, metric_areas="watershed")
attrs.head()

,COMID,WSAREASQKM,CLAYWS,SANDWS,ELEVWS,WETINDEXWS,BFIWS,KFFACTWS,AGKFFACTWS,RUNOFFWS,...,PCTURBMD2019WS,PCTHBWET2019WS,PCTCONIF2019WS,PCTURBOP2019WS,PCTSHRB2019WS,PCTCROP2019WS,PCTHAY2019WS,PCTMXFST2019WS,PCTGRS2019WS,PCTURBHI2019WS
0,400496,808.0020,34.487492,24.813596,346.2187,746.4028,None,NaN,NaN,379.1795,...,0.28,0.08,0.73,4.24,1.89,0.00,37.90,4.28,1.82,0.11
1,487460,158.6061,21.920000,23.720000,469.7755,580.5784,None,0.3100,0.0003,519.3887,...,0.21,0.00,0.04,3.73,4.20,0.00,0.38,7.37,2.13,0.02
2,789206,231.0615,21.042916,55.515971,69.9478,859.7173,None,0.1788,0.0152,727.5835,...,0.04,0.08,62.59,2.32,4.43,4.44,3.78,4.88,1.55,0.00
3,916821,65.1681,13.979957,49.745531,3602.2146,634.8088,None,0.2507,0.0000,118.5661,...,0.00,0.91,29.74,0.01,29.04,0.00,0.00,0.07,17.95,0.00
4,1056599,493.3233,34.170098,35.867487,176.6208,782.1230,None,0.2595,0.0389,429.8387,...,0.19,0.13,28.40,4.50,3.41,0.02,16.29,10.79,4.00,0.06


Next, we compute some hydrological signatures for these stations using their streamflow observations and `hydrosignatures` package.

In [10]:
fdc_slope = hsg.compute_fdc_slope(qobs_df, (33, 67), True)
signatures = pd.DataFrame(fdc_slope, index=qobs_df.columns, columns=["fdc_slope"])
signatures["seasonality_index"] = hsg.compute_si_walsh(qobs_df)
signatures.head()

,fdc_slope,seasonality_index
station_id,,
USGS-01022500,0.028700,0.608596
USGS-01031500,0.031548,0.746419
USGS-01047000,0.027627,0.688896
USGS-01052500,0.026508,0.687848
USGS-01054200,0.025889,0.656485


In [11]:
signatures["comid"] = comids.set_index("identifier").comid